In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModel, AutoTokenizer

In [10]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
test_labels_df = pd.read_csv('data/test_labels.csv')

train_df = train_df[['id', 'comment_text', 'toxic']]
test_labels_df = test_labels_df[['id', 'toxic']]

test_df = pd.merge(test_df, test_labels_df, on='id', how='inner')
test_df = test_df[test_df['toxic'] != -1]

In [19]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained("bert-base-uncased")

tokenizer_config.json: 100%|██████████| 48.0/48.0 [00:00<00:00, 115kB/s]
config.json: 100%|██████████| 570/570 [00:00<00:00, 908kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.27MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 2.09MB/s]


RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
cannot import name 'split_torch_state_dict_into_shards' from 'huggingface_hub' (/home/romh/miniconda3/envs/other/lib/python3.10/site-packages/huggingface_hub/__init__.py)

In [ ]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)